# **Import Necessary Packages:**

In [1]:
! pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.30.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [2]:
!pip install lmqg
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.6/662.6 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
  Created

In [3]:
# Import necessary packages for the fine-tuning process
import os                          # Operating system functionalities
import torch                       # PyTorch library for deep learning
from datasets import load_dataset  # Loading datasets for training
from transformers import (
    AutoModelForCausalLM,          # AutoModel for language modeling tasks
    AutoTokenizer,                # AutoTokenizer for tokenization
    BitsAndBytesConfig,           # Configuration for BitsAndBytes
    HfArgumentParser,             # Argument parser for Hugging Face models
    TrainingArguments,            # Training arguments for model training
    pipeline,                     # Creating pipelines for model inference
    logging,                      # Logging information during training
)
from peft import LoraConfig, PeftModel  # Packages for parameter-efficient fine-tuning (PEFT)
from trl import SFTTrainer

In [4]:
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
from datasets import load_dataset
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

# **Data preprocessing**

**Load Dataset:**

**Here Load the QA pairs part1**

In [5]:
import fitz  # PyMuPDF
import re
import json

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a given PDF file.

    Parameters:
    pdf_path (str): The path to the PDF file.

    Returns:
    str: The extracted text from the PDF.
    """
    # Open the PDF file
    doc = fitz.open(pdf_path)

    text = ""
    # Iterate through all the pages
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    return text

def parse_qa(text):
    """
    Parses text to extract questions and answers.

    Parameters:
    text (str): The extracted text containing Q&A pairs.

    Returns:
    list: A list of dictionaries with 'Question' and 'Answer' keys.
    """
    # Define regex pattern to find questions and answers
    pattern = re.compile(r"Question: (.*?)Answer: (.*?)(?=Question:|$)", re.DOTALL)

    # Find all matches
    matches = pattern.findall(text)

    # Create a list of dictionaries for each Q&A, replacing \n with spaces
    qa_pairs = [{"Question": match[0].strip(), "Answer": match[1].strip().replace('\n', ' ')} for match in matches]

    return qa_pairs


# Path to your PDF file
pdf_path = "/content/Large Language Modelling Internship - Take-Home Assignment - A Question-Answering Agent - Fall 2024.pdf"

# Extract text from PDF
text = extract_text_from_pdf(pdf_path)

# Parse extracted text for Q&A pairs
qa_pairssssssss = parse_qa(text)

# Save the Q&A pairs to a JSON file
json_path = "qa_pairs_part1.json"
with open(json_path, "w", encoding="utf-8") as json_file:
    json.dump(qa_pairssssssss, json_file, ensure_ascii=False, indent=4)

print(f"Q&A pairs have been saved to {json_path}")

Q&A pairs have been saved to qa_pairs_part1.json


**Here Load the QA pairs Part2 (Article 4-chapter )**

In [6]:
import json
from lmqg import TransformersQG

# Initialize the TransformersQG model for English
model = TransformersQG(language="en")

# Define the first context
context1 = ("Regulatory status of products: Without prejudice to Article 2(2) of Directive 2001/83/EC, "
            "upon a duly substantiated request of a Member State, the Commission shall, after consulting the "
            "Medical Device Coordination Group established under Article 103 of this Regulation (‘MDCG’), "
            "by means of implementing acts, determine whether or not a specific product, or category or group "
            "of products, falls within the definitions of ‘medical device’ or ‘accessory for a medical device’. "
            "Those implementing acts shall be adopted in accordance with the examination procedure referred to "
            "in Article 114(3) of this Regulation.")

# Generate questions and answers from the first context
qa_pairs1 = model.generate_qa(context1)

# Define the second context
context2 = ("The Commission may, on its own initiative, after consulting the MDCG, decide, by means of "
            "implementing acts, on the issues referred to in paragraph 1 of this Article. Those implementing acts "
            "shall be adopted in accordance with the examination procedure referred to in Article 114(3). "
            "The Commission shall ensure that Member States share expertise in the fields of medical devices, "
            "in vitro diagnostic medical devices, medicinal products, human tissues and cells, cosmetics, "
            "biocides, food and, if necessary, other products, in order to determine the appropriate regulatory "
            "status of a product, or category or group of products.")

# Generate questions and answers from the second context
qa_pairs2 = model.generate_qa(context2)

# Define the third context for regulatory status consultation
context3 = ("When deliberating on the possible regulatory status as a device of products involving medicinal products, "
            "human tissues and cells, biocides or food products, the Commission shall ensure an appropriate level of "
            "consultation of the European Medicines Agency (EMA), the European Chemicals Agency (ECHA) and the "
            "European Food Safety Authority (EFSA), as relevant.")

# Generate questions and answers from the third context
qa_pairs3 = model.generate_qa(context3)

# Create a list to hold all question-answer pairs
all_qa_pairs = []

# Function to process QA pairs
def process_qa_pairs(qa_pairs, all_pairs):
    if qa_pairs:
        first_qa_pair = qa_pairs[0]  # Get the first question-answer pair
        question = first_qa_pair[0]   # Unpack the question
        answer = first_qa_pair[1]     # Unpack the answer
        formatted_answer = f"According to Chapter 1, Article 4, {answer}"

        # Append the question and formatted answer to the list
        all_pairs.append({
            "Question": question,
            "Answer": formatted_answer
        })

# Process all sets of QA pairs
process_qa_pairs(qa_pairs1, all_qa_pairs)
process_qa_pairs(qa_pairs2, all_qa_pairs)
process_qa_pairs(qa_pairs3, all_qa_pairs)

# Save all question-answer pairs to a single JSON file
with open('qa_pairs_part2.json', 'w') as json_file:
    json.dump(all_qa_pairs, json_file, indent=4)

print("All question and answers saved to qa_pairs_part2.json.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

100%|██████████| 1/1 [00:00<00:00, 207.34it/s]

All question and answers saved to qa_pairs_part2.json.


**Here combined Both QA_pairs_part1 & QA_pairs_part2**

In [7]:
import json

# Paths to the JSON files
json_path1 = "qa_pairs_part1.json"
json_path2 = "qa_pairs_part2.json"

# Read the contents of the first JSON file
with open(json_path1, "r", encoding="utf-8") as json_file1:
    qa_pairs_part1 = json.load(json_file1)

# Read the contents of the second JSON file
with open(json_path2, "r", encoding="utf-8") as json_file2:
    qa_pairs_part2 = json.load(json_file2)

# Combine the contents of both JSON files
combined_qa_pairs_part2 = qa_pairs_part1 + qa_pairs_part2

# Save the combined data to a new JSON file
combined_json_path = "combined_qa_pairs_part2.json"
with open(combined_json_path, "w", encoding="utf-8") as json_file:
    json.dump(combined_qa_pairs_part2, json_file, ensure_ascii=False, indent=4)

print(f"Combined Q&A pairs have been saved to {combined_json_path}")

Combined Q&A pairs have been saved to combined_qa_pairs_part2.json


In [10]:
# Load your JSON dataset
dataset_path = "/content/combined_qa_pairs_part2.json"
data = load_dataset("json", data_files={"train": dataset_path})

# Check if the dataset is correctly loaded
print("Dataset loaded successfully. Available splits:", data)

# Ensure the 'train' split is available and not empty
if "train" not in data or len(data['train']) == 0:
    raise ValueError("The 'train' split is missing or empty in the dataset.")

# Print a sample data point to verify
print("Sample data point:", data["train"][0])

Dataset loaded successfully. Available splits: DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 12
    })
})
Sample data point: {'Question': 'What does apply to MDR/the regulation?', 'Answer': 'According to Chapter 1, Article 1, Section 1 of the MDR, this Regulation lays down rules concerning the placing on the market, making available on the market, or putting into service medical devices for human use and accessories for such devices in the European Union. This Regulation also applies to clinical investigations concerning medical devices and accessories in the European Union.'}


**Split Dataset:**

In [11]:
# Split the dataset into train and test sets (70-30 split)
data = data["train"].train_test_split(test_size=0.3)
train_data = data["train"]
test_data = data["test"]

# Print sizes of the splits
print(f"Training data size: {len(train_data)}, Test data size: {len(test_data)}")

Training data size: 8, Test data size: 4


**Generate and Tokenize Prompts:**

In [12]:
# Define the generate_prompt function
def generate_prompt(data_point):
    # Replace with your actual prompt generation logic
    return f"Question: {data_point['Question']} Answer: {data_point['Answer']}"

# Define the generate_and_tokenize_prompt function
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(
        full_prompt,
        padding='max_length',  # Ensure all sequences are of the same length
        truncation=True,       # Truncate sequences that are too long
        max_length=512,        # Set a max length for padding/truncation
        return_tensors='pt'    # Return PyTorch tensors
    )
    return {
        'input_ids': tokenized_full_prompt['input_ids'].squeeze(0),
        'attention_mask': tokenized_full_prompt['attention_mask'].squeeze(0)
    }

**Loading the Base model**

In [13]:
# Load the base model and tokenizer.
model_name = "NousResearch/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

**Load Tokenizer:**

In [14]:
# Generate and tokenize prompts without batching for training and testing sets
train_data = train_data.map(generate_and_tokenize_prompt, batched=False)
test_data = test_data.map(generate_and_tokenize_prompt, batched=False)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

**Remove Unnecessary Columns:**

In [15]:
# Remove unnecessary columns
train_data = train_data.remove_columns(data["train"].column_names)
test_data = test_data.remove_columns(data["test"].column_names)

**Load Model with Quantization:**

In [16]:
# Load the model with quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="fp4",
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)  #base model
model

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


# **Defining LORA and Training configuration**

**Prepare Model for k-bit Training and Attach LoRA:**

In [17]:
# Prepare the model for k-bit training and attach LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)

# Prepare the model for training
model = prepare_model_for_kbit_training(model)
peft_model = get_peft_model(model, lora_config)  # PEFT (fine tuned model saved in this variable)

In [18]:
def print_trainable_parameters(peft_model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in peft_model.parameters())
    print(f"Trainable parameters: {trainable_params} / {total_params} ({trainable_params / total_params:.2%})")

print_trainable_parameters(peft_model)

Trainable parameters: 8388608 / 3508801536 (0.24%)


**Set Up Training Arguments:**

In [19]:
# Set up the training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir="trained_model",
    max_steps=10,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    remove_unused_columns=False,
    report_to="tensorboard",
)

**Create Data Collator:**

In [20]:
# Create data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

**Set Up the Trainer & Training:**

In [21]:
# Set up the Trainer
trainer = Trainer(
    model=peft_model,
    train_dataset=train_data,
    args=training_args,
    data_collator=data_collator,
)

model.config.use_cache = False

# Start training
trainer.train()

output_dir="trained_model"
#save the Fine tuned model
trainer.save_model(output_dir)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.177200
2,2.239600
3,2.176600
4,2.182300
5,2.171200
6,1.835400
7,1.725100
8,1.879700
9,1.714200
10,1.770300


# **Saving the Fine tuned PEFT model**

In [22]:
peft_model.save_pretrained("peft final model_2")  #fine tuned  peft model
peft_model

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_proj): Li

In [23]:
tokenizer.save_pretrained("peft final model_2")

('peft final model_2/tokenizer_config.json',
 'peft final model_2/special_tokens_map.json',
 'peft final model_2/tokenizer.model',
 'peft final model_2/added_tokens.json',
 'peft final model_2/tokenizer.json')

# **Loading the PEFT trained model**

In [24]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)

PEFT_MODEL = "peft final model_2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="fp4",
)

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
model

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (v_proj): Li

# **Inferencing the loaded the peft model**

In [36]:
# Function to remove redundant sentences
def remove_redundancy(text):
    sentences = text.split('. ')
    seen = set()
    result = []
    for sentence in sentences:
        if sentence not in seen:
            result.append(sentence)
            seen.add(sentence)
    return '. '.join(result)

# Input from user
text_input = input('Enter the Question to chatbot: ')

# Tokenize input prompt
prompt = text_input
encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")

# Configure generation parameters
generation_config = model.generation_config
generation_config.max_new_tokens = 200          # Limit the number of tokens to a reasonable length
generation_config.temperature = 0.7            # Adjust temperature to make responses less repetitive
generation_config.top_p = 0.6                  # Further restrict token selection to reduce repetition
generation_config.repetition_penalty = 1.2     # Add a repetition penalty to avoid repeating phrases
generation_config.num_return_sequences = 1     # Return only one sequence
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.no_repeat_ngram_size = 2     # Avoid repeating phrases
generation_config.min_length = 10              # Ensure a minimum length for the answer

# Generate the response
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )

# Decode and process the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
clean_response = remove_redundancy(response)
print(clean_response)

Enter the Question to chatbot: What does apply to MDR/the regulation?
What does apply to MDR/the regulation?
 Hinweis: Die Anforderungen an die Zulassung von Medizinprodukten nach dem Medizinal- und Arzneimittelgesetz (MAHV) sind nicht Teil des Musterabkommens. Sie gelten für alle Medikamente, also auch solche, die im Rahmen der Verordnung über den Verkehr mit gefährlichen Stoffen (Gefahrstoffverkehrsverordnung - GSV), in Kraft getreten am 16.07.2013, als Gefahrgut eingestuft werden müssen.
The European Commission has published the new version of Annex II on clinical investigations for medical devices and IVDs under Article 5(4). The revision is based on a proposal from the Medical Devices Coordination Group (MDCG) which was adopted by the MDCG at its meeting on 9 March 2o18. It entered into force on April 


**Model Evaluation**

In [29]:
generation_config.max_new_tokens = 512

# Set the model to evaluation mode
model.eval()

# Initialize a list to store predictions
all_predictions = []

for i in range(len(test_data)):
    input_ids = torch.tensor(test_data[i]['input_ids']).unsqueeze(0).to(DEVICE)
    attention_mask = torch.tensor(test_data[i]['attention_mask']).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=200)

    # Decode the output to a readable format
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    all_predictions.append(prediction)

# Retrieve the true labels (assuming they are in 'text' or another key)
true_labels = [item['input_ids'] for item in test_data]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Input length of input_ids is 512, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 512, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 512, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
Input length of input_ids is 512, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `ma

In [30]:
print("True labels:", true_labels)
print("All predictions:", all_predictions)

True labels: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [31]:
print(f'True labels: {true_labels}')
print(f'Type of true_labels: {type(true_labels)}')

True labels: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [33]:
from nltk.translate.bleu_score import sentence_bleu

# Load your tokenizer (adjust to your specific model)
tokenizer = AutoTokenizer.from_pretrained("peft final model_2")

# Convert token IDs back to strings for true labels
true_labels_str = [tokenizer.decode(label, skip_special_tokens=True) for label in true_labels]

# Calculate BLEU score for each prediction
bleu_scores = []
for true, pred in zip(true_labels_str, all_predictions):
    # Calculate BLEU score for the current pair of true and predicted labels
    bleu_score = sentence_bleu([true.split()], pred.split())
    bleu_scores.append(bleu_score)

# Average BLEU score across the dataset
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f'Average BLEU Score: {avg_bleu_score}')

Average BLEU Score: 0.9806586134490999
